In [75]:
from matplotlib.pyplot import draw, fill
import pyrender
import trimesh
from trimesh import remesh
import pandas as pd
import numpy as np
from trimesh import Trimesh
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import os
import matplotlib.mlab as mlab
import seaborn as sns
import skeletor as sk
from view_data import view_mesh
import copy
from sklearn.preprocessing import MinMaxScaler
from project_statistics import get_outliers
from create_dataset import load_dataset,dir_to_sorted_file_list
from trimesh.repair import fill_holes
from trimesh.points import PointCloud
import ast
import random

In [217]:
data_normalized = pd.read_csv('fixed_features_df.csv')

In [218]:
data_normalized.describe()

,id,surface_area,compactness,sphericity,volume,diameter,rectangulairty,eccentricity
count,1814.000000,1814.000000,1814.000000,1814.000000,1814.000000,1814.0,1814.000000,1.813000e+03
mean,906.500000,1.682419,23.605474,4.290284,0.131645,1.0,0.481050,9.637006e+02
std,523.801012,1.759210,523.212320,29.467328,0.129171,0.0,0.250417,3.638733e+04
min,0.000000,0.029785,0.001066,0.000045,0.008233,1.0,0.010114,8.526999e-13
25%,453.250000,0.635022,0.817415,0.170113,0.040416,1.0,0.332857,2.249250e-01
50%,906.500000,1.166979,2.058936,0.485688,0.083667,1.0,0.440785,1.131218e+00
75%,1359.750000,2.120572,5.878437,1.223369,0.185493,1.0,0.578376,4.587717e+00
max,1813.000000,34.635823,22109.240816,937.812589,0.994923,1.0,3.839250,1.541949e+06


In [113]:
data_normalized.head()

,id,surface_area,compactness,sphericity,volume,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,-0.583110,-0.044830,0.071331,-0.039330,1.0,-0.109684,-0.026311,[ 3 22 100 180 262 512 443 405],[11871 11335 8000 5736 4745 4117 2946 1092],[ 2283 6080 5658 8386 15850 5245 2374 3966],[ 4717 9739 10818 11171 7455 3862 1498 582],[10272 7834 7009 6138 4530 3649 2826 7584],[ 6056 13523 13427 9884 4899 1575 418 60]
1,1,-0.640930,-0.044654,-0.009112,-0.415324,1.0,0.096999,-0.026489,[ 98 109 211 118 209 68 546 631],[12300 10978 7711 6014 4780 4018 2915 1136],[ 6157 12722 9005 5983 8014 5172 1814 985],[ 7481 14871 13645 8298 3089 1411 893 164],[18453 14253 8239 4427 2323 1130 587 440],[ 4990 16328 16309 8776 2874 501 66 8]
2,2,-0.603237,-0.044844,0.081945,-0.095674,1.0,-0.043276,-0.026263,[ 31 75 272 367 246 226 164 603],[12640 11734 7240 5179 4588 4207 3076 1167],[ 7827 10449 6542 9166 9360 2545 2115 1827],[ 6934 13206 13607 8837 4741 1788 579 139],[14598 11994 7919 5313 3757 2413 1509 2328],[ 5462 17399 14909 8062 3029 799 147 24]
3,3,-0.520618,-0.044900,0.137635,0.392969,1.0,-0.468705,-0.026398,[ 22 46 89 66 40 170 517 904],[10285 11658 9543 6547 4860 3779 2342 835],[2460 4632 4517 9448 8498 6270 5443 8581],[3095 7970 9794 9944 8724 6559 3132 631],[ 6569 7394 6728 5699 4730 3883 3171 11675],[ 3858 13091 15422 10617 4907 1543 365 46]
4,4,-0.724641,-0.044896,0.133228,-0.475577,1.0,-0.306206,-0.026249,[134 230 430 244 369 312 191 190],[12136 11576 7618 5606 4577 3858 3095 1383],[ 8807 9513 13040 8791 4435 1811 1770 1682],[ 5219 12768 13255 9502 5448 2448 1009 200],[14164 12684 8169 5446 3570 2289 1331 2196],[ 3794 13542 15441 10278 4652 1636 434 72]


In [114]:
shape_descriptor_normalized = data_normalized.copy()
for column in data.columns[8:]:
    shape_descriptor_column = data[column]
    print(shape_descriptor_column)
    for row in shape_descriptor_column:
        shape_descriptor_column
    break
    #normalized_feature_data = standardize(data[column])
    #data_normalized[column] = normalized_feature_data

0               [  3  22 100 180 262 512 443 405]
1               [ 98 109 211 118 209  68 546 631]
2               [ 31  75 272 367 246 226 164 603]
3               [ 22  46  89  66  40 170 517 904]
4               [134 230 430 244 369 312 191 190]
                          ...                    
1809    [ 178   21  107   97   19   63  576 1065]
1810            [149 257 642 288  78 127  72  37]
1811            [ 16  58 135 254 330 386 536 178]
1812            [298 499 554 228  81  82 115  62]
1813    [  61  381    0   13   59  182 1151   76]
Name: curvature, Length: 1814, dtype: object


In [84]:
shape_descriptor_column/np

0               [  3  22 100 180 262 512 443 405]
1               [ 98 109 211 118 209  68 546 631]
2               [ 31  75 272 367 246 226 164 603]
3               [ 22  46  89  66  40 170 517 904]
4               [134 230 430 244 369 312 191 190]
                          ...                    
1809    [ 178   21  107   97   19   63  576 1065]
1810            [149 257 642 288  78 127  72  37]
1811            [ 16  58 135 254 330 386 536 178]
1812            [298 499 554 228  81  82 115  62]
1813    [  61  381    0   13   59  182 1151   76]
Name: curvature, Length: 1814, dtype: object

In [122]:
string = shape_descriptor_column[0]

In [116]:
data_normalized.to_pickle('data_normalized.pkl')

In [189]:
import pickle
with open('features_finally_corrected_df.pkl', 'rb') as f:
    x = pickle.load(f)

In [190]:
x = np.array(x['curvature'])

In [192]:
x

array([array([  3,  22, 100, 180, 262, 512, 443, 405], dtype=int64),
       array([ 98, 109, 211, 118, 209,  68, 546, 631], dtype=int64),
       array([ 31,  75, 272, 367, 246, 226, 164, 603], dtype=int64)],
      dtype=object)

In [191]:
x.shape

(3,)

In [203]:
a = np.vstack(x)/np.sum(np.vstack(x))

In [206]:
b = a*1.1

In [204]:
a[0]

array([0.00050839, 0.00372818, 0.01694628, 0.0305033 , 0.04439925,
       0.08676496, 0.07507202, 0.06863244])

In [205]:
a[1]

array([0.01660735, 0.01847145, 0.03575665, 0.01999661, 0.03541773,
       0.01152347, 0.09252669, 0.10693103])

In [201]:
from scipy.stats import wasserstein_distance

In [207]:
wasserstein_distance(a[0], b[0])

0.004081935265209289

In [208]:
wasserstein_distance(a[1], b[1])

0.004215387222504663

In [209]:
wasserstein_distance(a[2], b[2])

0.0042026775122860575

In [210]:
wasserstein_distance(a[0], b[2])

0.01155736315878665

In [164]:
x['curvature']

0     [3, 22, 100, 180, 262, 512, 443, 405]
1    [98, 109, 211, 118, 209, 68, 546, 631]
2    [31, 75, 272, 367, 246, 226, 164, 603]
Name: curvature, dtype: object

In [151]:
s = string.replace('[','').replace(']','').split(' ')

In [153]:
s1 = ' '.join(s).split()

In [154]:
s1

['3', '22', '100', '180', '262', '512', '443', '405']

In [155]:
list(map(int, s1))

[3, 22, 100, 180, 262, 512, 443, 405]

In [ ]:
features_finally_corrected_df